<a href="https://colab.research.google.com/github/carlosjviola/MeLi-challenge/blob/main/MeLi_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MeLi** - Data Visualization & Analytics Challenge

Carlos José Viola - 2022/11 

[Ver especificaciones](https://drive.google.com/file/d/1CTPIC5OIZ58joVDMG9AYAEecKTCsRtlc/view?usp=share_link)

Supuestos / Consideraciones

1) El archivo tiene registros correspondientes al mes de enero de 2022 para el campo first_login_dt

2) No hay especificaciones para el campo week_year. Se considera corresponde a la semana del año del campo first_login_dt

3) Se realizan distintas operaciones de limpieza de datos

3) Los registros del 1 y 2 de enero vienen con la semana 52. De acuerdo a lo supuesto en el punto anterior se reemplazarán los valores.

4) Removemos 

**#Creamos un DataFrame de Pandas para analizar el archivo**

In [86]:
import pandas as pd
import numpy as np
from datetime import date

# Se crea una variable para almacenar nombre del archivo a analizar
archivoCsv = 'https://raw.githubusercontent.com/carlosjviola/MeLi-challenge/main/dataset_challenge_analytics.csv'

# Cargamos el archivo en un DataFrame de Pandas realizar el estudio
dfUsrOrig = pd.read_csv(archivoCsv)

#Vemos el tamaño del archivo (filas, columnas) y los primeras y últimas filas del mismo
print("Cantidad de filas y columnas")
print(dfUsrOrig.shape)

print("\nMuestra las primeras 10 filas")
print(dfUsrOrig.head(10))

print("\nMuestra las últimas 10 filas")
print(dfUsrOrig.head(10))

Cantidad de filas y columnas
(11500, 12)

Muestra las primeras 10 filas
   Unnamed: 0 first_login_dt  week_year         user_id  habito habito_dt  \
0           0     2022-01-30          4   MLB2883855850     0.0       NaN   
1           1     2022-01-30          4   MLB7059024930     0.0       NaN   
2           2     2022-01-07          1   MLB4163269880     0.0       NaN   
3           3     2022-01-07          1   MLB7349806870     0.0       NaN   
4           4     2022-01-07          1   MLB7239064190     0.0       NaN   
5           5     2022-01-01         52  MLB10318536730     0.0       NaN   
6           6     2022-01-01         52   MLB7546352300     0.0       NaN   
7           7     2022-01-16          2   MLB2785023500     0.0       NaN   
8           8     2022-01-16          2   MLB8395954300     0.0       NaN   
9           9     2022-01-16          2   MLB1610695500     0.0       NaN   

   activacion activacion_dt  setup    setup_dt  return   return_dt  
0          

In [87]:
#Removemos las primera columna que contiene un id que entiendo no vamos a utilizar
dfUsr = dfUsrOrig.drop(columns = dfUsrOrig.columns[0], axis = 1, inplace= False)

print("El nuevo DataFrame tiene una columna menos")
print(dfUsr.shape)


El nuevo DataFrame tiene una columna menos
(11500, 11)


**#La columna user_id debería contener valores únicos y es la base para el análisis**

In [88]:
#Vamos a analizar la columna user_id y ordenamos esa columna
dfUsr.sort_values('user_id', inplace= True)
print(dfUsr['user_id'].describe())

print("\nObservamos una muestra aleatoris de 50 registros")
pd.set_option('display.max_rows', None)
print(dfUsr['user_id'].sample(n=50))

count             11500
unique            11000
top       MLB7852379180
freq                  2
Name: user_id, dtype: object

Observamos una muestra aleatoris de 50 registros
9693       MLB3489362330
7598      MLB10605700240
3496      MLB10594402490
8283       MLB2110294460
3335      MLB10266530370
5937      MLB10156336020
8141       MLB1170480320
4347       MLB2274477160
6816      MLB10657042290
9164       MLB1611317540
3681       MLB5149774500
8885      MLB10565056080
7368       MLB8398073560
2284      MLB10549135110
8829       MLB4773716710
7088       MLB1603675440
8232      MLB10573229460
9519      MLB10415790950
2816       MLB1754306140
6009      MLB10211984500
8881       MLB3775394610
2575       MLB1484169470
167        MLB1576314000
5603       MLB2651073110
3615      MLB10509701870
1899       MLB4378277870
2193       MLB2100608490
8538      MLB10630107790
281        MLB2318835060
5967       MLB8369496460
7754        MLB554584920
8697      MLB10553285940
3654       MLB5877019680


In [89]:
#En algunos registros aparece '-1' al final lo que probablemente se trata de un error
#Reemplazamos los caracteres '-1' 
dfUsr['user_id'] = dfUsr['user_id'].str.replace('-1','')

#Vamos a verificar que el campo user_id tenga valores únicos
print("Cantidad total registros")
print(len(dfUsr['user_id']))

print("\nCandidad registros únicos")
print( len(dfUsr['user_id'].unique()) )

#Cómo han aparecido duplicados vamos a eliminarlos.
dfUsr.drop_duplicates(subset='user_id', inplace=True, keep='first')

print("\nNueva cantidad total registros sin duplicados")
print(len(dfUsr['user_id']))


Cantidad total registros
11500

Candidad registros únicos
10000

Nueva cantidad total registros sin duplicados
10000


**#Comenzamos el análisis de otras columnas**

In [90]:
# Convertimos la columna hábito en entero de acuerdo a las especificaciones
# antes llenamos con 0 los valores nulos en esta columna
display(dfUsr.dtypes)

dfUsr['habito'] = dfUsr['habito'].fillna(0.0).astype(int)
  
print('\nVerificamos los cambios')
display(dfUsr.dtypes)

first_login_dt     object
week_year           int64
user_id            object
habito            float64
habito_dt          object
activacion          int64
activacion_dt      object
setup               int64
setup_dt           object
return              int64
return_dt          object
dtype: object


Verificamos los cambios


first_login_dt    object
week_year          int64
user_id           object
habito             int64
habito_dt         object
activacion         int64
activacion_dt     object
setup              int64
setup_dt          object
return             int64
return_dt         object
dtype: object

In [91]:
# Convertimos en date los datos date-time en las columnas fecha. 
# Haciendo una inspección rápida del csv se observa que no hay horas

dfUsr['first_login_dt'] = pd.to_datetime(dfUsr['first_login_dt']).dt.date
dfUsr['habito_dt'] = pd.to_datetime(dfUsr['habito_dt']).dt.date
dfUsr['activacion_dt'] = pd.to_datetime(dfUsr['activacion_dt']).dt.date
dfUsr['setup_dt'] = pd.to_datetime(dfUsr['setup_dt']).dt.date
dfUsr['return_dt'] = pd.to_datetime(dfUsr['return_dt']).dt.date


In [92]:
# Como no está muy claro su contenido vamos a analizar la columna week_year
dfUsr['week_year'].agg(['mean', 'max', 'min'])

mean     4.3559
max     52.0000
min      1.0000
Name: week_year, dtype: float64

In [93]:
#Los datos son de enero, no puede haber registros con el valor 52 en semana
print("Cantidad de registros con error en semana")
print(len(dfUsr[dfUsr['week_year']==52]))

print(dfUsr[dfUsr['week_year']==52].sample(n=25))


Cantidad de registros con error en semana
350
     first_login_dt  week_year         user_id  habito   habito_dt  \
7893     2022-01-01         52   MLB8058073600       0         NaT   
8225     2022-01-02         52  MLB10359574110       0         NaT   
5304     2022-01-02         52  MLB10492195060       0         NaT   
7508     2022-01-01         52  MLB10465725250       0         NaT   
3872     2022-01-01         52   MLB5100960160       0         NaT   
18       2022-01-02         52  MLB10492545680       1  2022-01-19   
565      2022-01-01         52   MLB7376185410       0         NaT   
9592     2022-01-02         52   MLB4046329700       0         NaT   
4018     2022-01-02         52  MLB10478346580       0         NaT   
7390     2022-01-02         52    MLB189051670       0         NaT   
1708     2022-01-02         52   MLB4588634450       1  2022-01-08   
5995     2022-01-01         52   MLB8192620260       1  2022-01-16   
5139     2022-01-01         52   MLB80079446

In [99]:
#Vemos que los registros con semana 52 corresponden al 1 y 2 de enero
#Como son los primeros días del año quedaron con la última semana del anterior. 
#Debemos reemplazar 52 por 1
dfUsr.loc[dfUsr["week_year"] == 52, "week_year"] = 1

print("Cantidad de registros con error en semana")
print(len(dfUsr[dfUsr['week_year']==52]))


Cantidad de registros con error en semana
0


**#Ahora hacemos un control con las fechas. Las fechas en las columnas activacion, setup, habito y return deben ser mayores o iguales a la fecha de first_login**

In [100]:
#Creamos una nueva columna max_dt en el dataset con la mayor fecha entre activación, setup, habito y return
def calc_new_col(row):
    columnas = ['activacion_dt', 'setup_dt', 'habito_dt', 'return_dt']
    max_dt = date.min 
    for col in columnas:
        if row[col] is not pd.NaT:
          if row[col] > max_dt:
              max_dt = row[col]
    return max_dt

dfUsr["max_dt"] = dfUsr.apply(calc_new_col, axis=1)
print(dfUsr.head()) 
print(dfUsr.shape) 


     first_login_dt  week_year         user_id  habito   habito_dt  \
5561     2022-01-13          2  MLB10001480800       0         NaT   
9631     2022-01-06          1  MLB10001530370       0         NaT   
5035     2022-01-05          1  MLB10002487550       1  2022-01-18   
8479     2022-01-07          1  MLB10002855700       0         NaT   
3189     2022-01-15          2  MLB10003112290       0         NaT   

      activacion activacion_dt  setup    setup_dt  return   return_dt  \
5561           1    2022-01-13      0         NaT       0         NaT   
9631           1    2022-01-06      0         NaT       1  2022-01-13   
5035           1    2022-01-07      1  2022-01-08       1  2022-01-06   
8479           0           NaT      0         NaT       0         NaT   
3189           0           NaT      0         NaT       0         NaT   

          max_dt  
5561  2022-01-13  
9631  2022-01-13  
5035  2022-01-18  
8479  0001-01-01  
3189  0001-01-01  
(10000, 12)


In [101]:
print('Tamaño del dataset')
print(dfUsr.shape) 

#Creamos una serie con el índice de los registros con problemas de fecha
indexUsr = dfUsr[ dfUsr['max_dt'] < dfUsr['first_login_dt']].index
print('\nCantidad de filas con problemas de fecha')
print(indexUsr.shape) 

#Creamos un nuevo DataFrame borrando los registros con problemas detectados en el punto anterior
dfUsrFinal = dfUsr.drop(indexUsr)

print('\nResumen de archivo con los registros borrados')
print(dfUsrFinal.shape)

print(dfUsrFinal.head(15))

Tamaño del dataset
(10000, 12)

Cantidad de filas con problemas de fecha
(2119,)

Resumen de archivo con los registros borrados
(7881, 12)
     first_login_dt  week_year         user_id  habito   habito_dt  \
5561     2022-01-13          2  MLB10001480800       0         NaT   
9631     2022-01-06          1  MLB10001530370       0         NaT   
5035     2022-01-05          1  MLB10002487550       1  2022-01-18   
3103     2022-01-19          3  MLB10003554750       1  2022-02-08   
7609     2022-01-21          3  MLB10003698160       0         NaT   
2770     2022-01-02          1  MLB10004252630       0         NaT   
4929     2022-01-19          3  MLB10004759110       0         NaT   
2343     2022-01-26          4  MLB10005576530       0         NaT   
5950     2022-01-10          2  MLB10006217790       1  2022-01-23   
7859     2022-01-20          3  MLB10007151650       0         NaT   
857      2022-01-07          1  MLB10010599840       0         NaT   
4609     2022-01-17  

**#Ahora vamos a crear columnas con la cantidad de días que trancurren entre las fechas del archivo y la de forst_login**

In [106]:
dfUsrFinal['activacion_dias'] = (dfUsrFinal['activacion_dt']-dfUsrFinal['first_login_dt']).dt.days
dfUsrFinal['setup_dias'] = (dfUsrFinal['setup_dt']-dfUsrFinal['first_login_dt']).dt.days
dfUsrFinal['return_dias'] = (dfUsrFinal['return_dt']-dfUsrFinal['first_login_dt']).dt.days
dfUsrFinal['habito_dias'] = (dfUsrFinal['habito_dt']-dfUsrFinal['first_login_dt']).dt.days

print('Observamos el encabezado del nuevo DataFrame')
print(dfUsrFinal.head(15))

Observamos el encabezado del nuevo DataFrame
     first_login_dt  week_year         user_id  habito   habito_dt  \
5561     2022-01-13          2  MLB10001480800       0         NaT   
9631     2022-01-06          1  MLB10001530370       0         NaT   
5035     2022-01-05          1  MLB10002487550       1  2022-01-18   
3103     2022-01-19          3  MLB10003554750       1  2022-02-08   
7609     2022-01-21          3  MLB10003698160       0         NaT   
2770     2022-01-02          1  MLB10004252630       0         NaT   
4929     2022-01-19          3  MLB10004759110       0         NaT   
2343     2022-01-26          4  MLB10005576530       0         NaT   
5950     2022-01-10          2  MLB10006217790       1  2022-01-23   
7859     2022-01-20          3  MLB10007151650       0         NaT   
857      2022-01-07          1  MLB10010599840       0         NaT   
4609     2022-01-17          3  MLB10011098280       0         NaT   
156      2022-01-28          4  MLB1001172856

**#Grabamos los resultados del análisis en un nuevo archivo que se utilizará en el reporte**

In [107]:
dfUsrFinal.to_csv('resultado_final.csv')


# Nueva sección